In [8]:
using DynamicPolynomials

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
g=[1.0-sum(x.^2)] #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=0#ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=2*rand(n).-1# create a feasible solution
randx=rand(1)[1]*randx./sqrt(sum(randx.^2))


for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=1
Number of equality constraints: l=0


In [9]:
using ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [10]:

k=2

println("Relaxation order: k=",k)

Relaxation order: k=2


In [11]:
using ctpPOP

opt_val1=ctpPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=2
  Size of the largest block: s^max=21
  Number of equality trace constraints: zeta=127
Modeling time:
  0.007441 seconds (11.43 k allocations: 1.212 MiB)
iter=1   val=-3.723058261182444   gap=0.020775133027743427   feas=2.982416951811426
iter=2   val=-1.2712427163948798   gap=0.5558158031450235   feas=0.4975873537764451
iter=4   val=-1.3392131561376852   gap=0.4335926604191025   feas=0.47516154415675893
iter=8   val=-1.4326580589823255   gap=0.024484411555608144   feas=0.15068242731489367
iter=16   val=-1.4818836284606989   gap=0.01783653111394389   feas=0.03307129307286379
iter=32   val=-1.5264673008216985   gap=0.0071363773468949895   feas=0.007274184811025378
iter=64   val=-1.5378748953253205   gap=0.0019365466802161134   feas=0.0018402516871921426
iter=92   val=-1.5398432272403761   gap=0.0009837015216999422   feas=0.0008947375844657054
tol satisfies!!!

###############################

-1.5398432272403761

In [12]:
using ctpPOP

opt_val2=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 126             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

-1.5417031692794239

In [13]:
using ctpPOP

opt_val=ctpPOP.POP_dense_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=2
  Size of the largest block: s^max=21
  Number of equality trace constraints: zeta=127
Modeling time:
  0.008024 seconds (11.43 k allocations: 1.212 MiB)
---------------
| Parameters: |
---------------
n:       127
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.59615058D+00  WK= 0.4843D+00  QK= 0.2422D+00
 NIT=    2  NFE=    2  F= 0.33308285D+00  WK= 0.1122D+00  QK= 0.9080D-01
 NIT=    3  NFE=    3  F= 0.27197313D+00  WK= 0.4893D-01  QK= 0.3026D-01
 NIT=    4  NFE=    4  F= 0.25642872D+00  WK= 0.2058D-01  QK= 0.2862D-01
 NIT=    5  NFE=    5  F= 0.24852202D+00  WK= 0.2507D-02  QK= 0.4097D-02
 NIT=    6  NFE=    6  F= 0.24662019D+00  WK= 0.2365D-02  QK= 0.1303D-02
 Exit from LMBM:
 NIT=    7  NFE=    7  F= 0.24533549D+00  WK= 0.4545D-03  QK= 0.3756D-03  ITERM=  1
-----------
| Output: |
-----------
Termination:     1
N. iter

-1.5435532550989586

In [14]:
using ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.5479954692305942